In [1]:
import os
import shutil
import numpy as np
from mtcnn import MTCNN
import tensorflow as tf
from tqdm import tqdm
from retinaface import RetinaFace

# Set the path to the VGGFace2 dataset
orig_dir = "I:/balanced_vggface2"

# Set the path to the directory where you want to copy the selected images
new_dir = "E:/balanced_vggface2"

# Set the path to the text file to save the selected file paths
# txt_path = "E:/files.txt"

# Set the number of images to select per person
num_images = 30

# Create a list to store the selected file paths
selected_files = []

# Create the new directory if it doesn't exist
# if not os.path.exists(new_dir):
#     os.makedirs(new_dir)

# Initialize the MTCNN face detector
# detector = MTCNN()

In [2]:
import os
import random
import shutil
from tqdm import tqdm

# Set the paths to your image dataset folders
data_dir = "I:/balanced_vggface2"
train_dir = 'I:/balanced_vggface2/train'
test_dir = 'I:/balanced_vggface2/test'
val_dir = 'I:/balanced_vggface2/val'

# Set the number of images you want for each category
num_train = 20
num_test = 5
num_val = 5

# Get the list of classes in your dataset
classes = os.listdir(train_dir)

# Create the train, test, and val folders if they don't already exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# # Loop through each class and copy the images to the appropriate folders
# for class_name in tqdm(classes):
#     class_path = os.path.join(data_dir, class_name)
#     images = os.listdir(class_path)
#     random.shuffle(images)  # Shuffle the list of images
    
#     # Copy images to the train folder
#     for image in images[:num_train]:
#         src_path = os.path.join(class_path, image)
#         dst_path = os.path.join(train_dir, class_name, image)
#         os.makedirs(os.path.dirname(dst_path), exist_ok=True)
#         shutil.copy(src_path, dst_path)
    
#     # Copy images to the test folder
#     for image in images[num_train:num_train+num_test]:
#         src_path = os.path.join(class_path, image)
#         dst_path = os.path.join(test_dir, class_name, image)
#         os.makedirs(os.path.dirname(dst_path), exist_ok=True)
#         shutil.copy(src_path, dst_path)
    
#     # Copy images to the validation folder
#     for image in images[num_train+num_test:num_train+num_test+num_val]:
#         src_path = os.path.join(class_path, image)
#         dst_path = os.path.join(val_dir, class_name, image)
#         os.makedirs(os.path.dirname(dst_path), exist_ok=True)
#         shutil.copy(src_path, dst_path)


In [4]:
# import onnx
# import warnings
# from onnx_tf.backend import prepare
# import numpy as np
# from datetime import datetime
# import tensorflow as tf
# import os
# import cv2
# from tqdm import tqdm
# import PIL
# from PIL import Image

# onnx_model = onnx.load('F:/test/onnx_tensorflow/model.onnx')
# tf_rep = prepare(onnx_model) # Import the ONNX model to Tensorflow

In [3]:
from tqdm import tqdm
import os
# Set the path to the directory where you want to copy the selected images
new_dir = "I:/balanced_vggface2"

all_ids_dict = dict()

for id in tqdm(classes):
    if id in ['train', 'test', 'val']:
        continue
    all_ids_dict[id] = {'train':[], 'test':[], 'val':[]}
    for file in os.listdir(os.path.join(new_dir, 'train', id))[:num_train]:
        all_ids_dict[id]['train'].append(os.path.join(new_dir, 'train', id, file))
    for file in os.listdir(os.path.join(new_dir, 'test', id))[:num_test]:
        all_ids_dict[id]['test'].append(os.path.join(new_dir, 'test', id, file))
    for file in os.listdir(os.path.join(new_dir, 'val', id))[:num_val]:
        all_ids_dict[id]['val'].append(os.path.join(new_dir, 'val', id, file))

100%|██████████| 8907/8907 [00:06<00:00, 1342.22it/s]


In [7]:
import json
os.makedirs(os.path.join('.', 'vggface2'))
with open(os.path.join('.', 'vggface2', 'all_id_files.json'), 'w') as fp:
    json.dump(all_ids_dict, fp, indent=4)

In [8]:
import json
f = open(os.path.join('.', 'vggface2', 'all_id_files.json'), "r")

# Reading from file
all_ids_dict = json.loads(f.read())

# Closing file
f.close()

In [9]:
keys = list(all_ids_dict.keys())

In [10]:
import tensorflow as tf
import PIL
from PIL import Image
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch

dataset_path = 'I:/balanced_vggface2'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

os.makedirs(os.path.join(dataset_path, 'new_embeddings', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'new_embeddings', 'test'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'new_embeddings', 'val'), exist_ok=True)

img_size = 160 # 112
for d in tqdm(keys[:50000]):
    if os.path.isfile(os.path.join(dataset_path, 'new_embeddings', 'val', d + '.npz')):
        continue

    image_list = []
    for img_path in all_ids_dict[d]['train']:
        img = Image.open(img_path)
        x_train = tf.image.resize(np.array(img), (img_size, img_size), method="nearest")
        x_train = (tf.cast(x_train, tf.float32) - 127.5) / 128.
        x_train = tf.transpose(x_train, perm=[2, 0, 1])
        x_train = tf.expand_dims(x_train, 0)
        image_list.extend(x_train.cpu().numpy())

    for img_path in all_ids_dict[d]['test']: 
        img = Image.open(img_path)
        x_test = tf.image.resize(np.array(img), (img_size, img_size), method="nearest")
        x_test = (tf.cast(x_test, tf.float32) - 127.5) / 128.
        x_test = tf.transpose(x_test, perm=[2, 0, 1])
        x_test = tf.expand_dims(x_test, 0)
        image_list.extend(x_test.cpu().numpy())

    for img_path in all_ids_dict[d]['val']: 
        img = Image.open(img_path)
        x_val = tf.image.resize(np.array(img), (img_size, img_size), method="nearest")
        x_val = (tf.cast(x_val, tf.float32) - 127.5) / 128.
        x_val = tf.transpose(x_val, perm=[2, 0, 1])
        x_val = tf.expand_dims(x_val, 0)
        image_list.extend(x_val.cpu().numpy())
        
    # id_emb = tf_rep.run(np.array(image_list))._0
    id_emb = tuple(torch.Tensor(image_list))
    id_emb = torch.stack(id_emb).cuda()
    id_emb = resnet(id_emb).detach().cpu().numpy()
    np.savez_compressed(os.path.join(dataset_path, 'new_embeddings', 'train', d + '.npz'), res=id_emb[:20])
    np.savez_compressed(os.path.join(dataset_path, 'new_embeddings', 'test', d + '.npz'), res=id_emb[20:25])
    np.savez_compressed(os.path.join(dataset_path, 'new_embeddings', 'val', d + '.npz'), res=id_emb[25:])

100%|██████████| 8907/8907 [2:23:16<00:00,  1.04it/s]  
